In [1]:
import pandas as pd
proc_df = pd.read_csv("../data/fake_job_postings.csv")

In [2]:
proc_df = proc_df.where(pd.notnull(proc_df), ' ')
proc_df['aggr_post'] = proc_df["company_profile"] + " " + proc_df["description"]+ " " + proc_df["requirements"]+ " " + proc_df["benefits"]

In [3]:
import langid

def detect_lang(x):
    code,_ = langid.classify(x)
    
    return code

proc_df = proc_df[proc_df['aggr_post'].apply(lambda x: detect_lang(x) == 'en')]

import re
import string

def clean_text(text):
    text = text.lower()                                              # make the text lowercase
    text = re.sub('\[.*?\]', '', text)                               # remove text in brackets
    text = re.sub('http?://\S+|www\.\S+', '', text)                  # remove links
    text = re.sub('https?://\S+|www\.\S+', '', text)                 # remove links
    text = re.sub('<.*?>+', '', text)                                # remove HTML stuff
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # get rid of punctuation
    text = re.sub('\n', '', text)                                    # remove line breaks
    text = re.sub('\w*\d\w*', '', text)                             # remove anything with numbers, if you want
    text = re.sub(r'[^\x00-\x7F]+',' ', text)                       # remove unicode
    return text

proc_df['aggr_post'] = proc_df['aggr_post'].apply(lambda x: clean_text(x))

proc_df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,aggr_post
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0,were and weve created a groundbreaking and aw...
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0,seconds the worlds cloud video production se...
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0,valor services provides workforce solutions th...
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,our passion for improving quality of life thro...
4,5,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,spotsource solutions llc is a global human cap...


In [4]:
ls = proc_df['aggr_post'].values

In [5]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer()
#計算詞語出現的次數
X = vectorizer.fit_transform(ls)
#獲取corpus所有文本的關鍵詞 
word = vectorizer.get_feature_names()
# print(word)
#查看結果
#print(X.toarray())
count = X.toarray()
count.shape

(17693, 145604)

In [6]:
#TF
import numpy as np
TF = np.zeros(count.shape[1])
for i in range(count.shape[0]):
    TF += count[i]
TF = np.log(TF)+1

In [7]:
#IDF
分子 = count.shape[0]
tmp = count
tmp[tmp>=1] = 1
q = np.zeros(count.shape[1])
for i in range(count.shape[0]):
    q += tmp[i]
q = 分子/q

In [8]:
#計算TF*IDF
TF*q

array([ 3577.98677453,  2781.08518835, 17693.        , ...,
        2781.08518835,  2529.8405106 , 12376.91574113])

In [9]:
d = {'number': word, 'TF_IDF': TF*q}
df_tmp = pd.DataFrame(d)

In [10]:
sorted(set(list(df_tmp.TF_IDF.values)),reverse=True)

[72382.81412726869,
 67821.04369638664,
 66748.41226258845,
 65606.5322081014,
 64385.84133288277,
 61658.45335469909,
 60118.96106162157,
 58432.63805034365,
 56568.49444680973,
 54484.559196941336,
 52121.98826723566,
 49394.600289051974,
 46168.78498469654,
 42220.70613129423,
 40548.100289051974,
 37130.74722340487,
 36191.407063634346,
 34416.17375622842,
 33910.52184819332,
 33374.206131294224,
 32803.2661040507,
 32192.920666441383,
 31537.324490783485,
 30829.226677349547,
 30059.480530810786,
 29956.853065647116,
 29216.319025171826,
 28604.55383901869,
 28284.247223404866,
 27242.279598470668,
 26060.99413361783,
 25335.413890071533,
 24881.523323131027,
 24697.300144525987,
 24640.768806782067,
 24389.76638747815,
 23853.24516354684,
 23262.98628613061,
 23084.39249234827,
 22944.115837485613,
 22607.014565462214,
 22249.47075419615,
 21868.844069367136,
 21546.539871498328,
 21461.947110960922,
 21110.353065647116,
 21024.88299385566,
 20552.817784899697,
 20513.20298100973

In [11]:
sum(df_tmp.TF_IDF.values > 17693.0)

3682

In [12]:
df_tmp = df_tmp.sort_values(by=['TF_IDF'],ascending=False)

In [13]:
#df_tmp.to_csv('final.csv',encoding='UTF-8')

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(vocabulary=df_tmp[:3682].number.values)
x = vectorizer.fit_transform(ls)
word = vectorizer.get_feature_names()
df1 = pd.DataFrame(x.toarray())
df1.columns = word
# df1 是原來datasets 取3682個高的TFIDF字做出來的字的datasets 也是有17693個rows , 3682個columns

In [15]:
# df1

In [16]:
proc_df = proc_df.reset_index()

In [17]:
proc_df = proc_df[['department', 
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent']]

In [18]:
proc_df.columns

Index(['department', 'telecommuting', 'has_company_logo', 'has_questions',
       'employment_type', 'required_experience', 'required_education',
       'industry', 'function', 'fraudulent'],
      dtype='object')

In [19]:
data_dum = pd.get_dummies(proc_df)
df2 = pd.DataFrame(data_dum)

In [20]:
df = pd.concat([df2,df1],axis=1)

In [21]:
df

,telecommuting,has_company_logo,has_questions,fraudulent,department_,department_ Corporate Shared Services,department_ Lower Level Management,department_ Marketing,department_ Moni Technologies,department_ R&D,...,dxw,marinetraffic,ticomix,vault,katapult,dmp,scorebig,snapstream,pixlee,granttree
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.to_csv('../data/fake_job_postings_TFIDF.csv')